In [31]:
# Importeer benodigde modules
import json
import cv2
import os

# **Unsupported:** Labelbox To Yolo
Dit script is geschreven tijdens de overgang van custom vision modellen naar yolo modellen. Het annoteren van afbeeldingen wert gedaan met behulp van [Labelbox](https://labelbox.com/) een gratis online annotatie programma waar je met meerdere mensen samen kan annoteren. De annotaties van Labelbox konden gedownload worden als een json file met daarin labels voor alle foto's van een project in coco notatie. Dit script is bedoeld om dit Json bestand te converteren naar verschillende txt bestanden met ieder de labels van één afbeelding in YOLO notatie.

---
<blockquote style="border-left: 5px solid red; padding-left: 10px;">

## **Attentie!** 
<code> Update onderstaande cell voordat je de het script runt </code></blockquote>
 

In [ ]:
# Output map waar de nieuwe YOLO labels worden opgeslagen.
OutputLocatie = r"pad/naar/jouw/output/folder"
# Bijvoorbeeld: r"pad/naar/jouw/output/folder"

# Input map met de foto's die Labels nodig hebben.
InputFotos = r"pad/naar/jouw/folder"
# Bijvoorbeeld: r"pad/naar/jouw/folder"

# Input map met de Json file met labels die je gedownload hebt van Labelbox.
JsonFile = r"pad/naar/jouw/Json/labels.json"
# Bijvoorbeeld: r"pad/naar/jouw/Json/labels.json"

---
## **Code**

In [32]:
def LabelBoxToYolo(ImageMap, LabelMap, Json):
    file = open(Json)
    JsonData = json.load(file)
    file.close()

    # Functie om een nieuwe folder aan te maken. 
    # Om het overschrijven van eerdere runs te voorkomen checkt de functie of de folder bestaat.
    # Bestaat de folder al dan word het nummer verhoogt to dat er een outputfolder gevonden word die nog niet bestaat. 
    # Deze word aangemaakt en terug gegeven
    def create_label_Location(location=LabelMap, JsonData=JsonData):
        ProjectNaam = JsonData[0]["Project Name"]
        i = 0
        while True:
            dirname = f"{ProjectNaam}, Labels({i})" if i > 0 else f"{ProjectNaam}, Labels"
            dirpath = os.path.join(location, dirname)
            if not os.path.exists(dirpath):
                os.makedirs(dirpath)
                break
            i += 1
        return dirpath

    LabelFiles = create_label_Location()

    # Loopt door het json file en maakt een set van alle gevonden Labelnamen/object title's
    objects = set()
    for photo in JsonData:
        for Object in photo["Label"]["objects"]:
            objects.add(Object["title"])

    # Maakt een Locatie aan om de het Labels.txt bestant op te vullen.
    # Loopt daarna over alle gevonden object titels en filter de unieke labels uit en slaat ze op in Labels.txt 
    LabelList = os.path.join(LabelFiles, "Labels.txt")
    objects.sorted()
    for name in objects:
        with open(LabelList, "a+") as a:
            a.write(name + "\n")

    # loopt door alle gedownloade foto's heen en filtert een voor een de bijbehorende labels uit
    for Image in os.listdir(ImageMap):
        AnnoName = Image.replace(".jpg", ".txt").replace(".JPG", ".txt")
        AnnoLocation = os.path.join(LabelFiles, AnnoName)
        ImageLocation = os.path.join(ImageMap, Image)

        # checkt of maakt het label bestand aan en verzekert dat het leeg is
        with open(AnnoLocation, "w+") as a:
            a.write("")

        # Haalt de afmetingen van de foto op om later te kunnen gebruiken voor het omrekenen van de coördinaten
        if os.path.exists(ImageLocation):
            img = cv2.imread(ImageLocation)
            img_y, img_x = img.shape[:2]

        # Zoekt naar de annotaties van de foto in het json bestand    
        for photo in JsonData:
            if Image in photo["External ID"]:
                for Box in photo["Label"]["objects"]:
                    # Gebruikt het eerder gemaakte Labels.txt om de "titles" om te zetten in label nummers(YOLO standaard)
                    with open(LabelList, 'r') as b:
                        lines = b.readlines()
                        for i, lines in enumerate(lines):
                            if Box["title"] in lines:
                                yoloL = i
                    # Rakend de COCO annotaties van LabelBox om naar de gewenste YOLO notaties 
                    # en voegt het label in de correcte volgorde toe aan het nieuwe annotatie bestand
                    yoloX = (Box["bbox"]["left"] + (Box["bbox"]["width"]/2))/img_x
                    yoloY = (Box["bbox"]["top"] + (Box["bbox"]["height"]/2))/img_y
                    yoloW = Box["bbox"]["width"] / img_y
                    yoloH = Box["bbox"]["height"] / img_x
                    with open(AnnoLocation, "a") as c:
                        c.write(f"{yoloL} {yoloX} {yoloY} {yoloW} {yoloH}\n")

LabelBoxToYolo(ImageMap=InputFotos, LabelMap=OutputLocatie, Json=JsonFile)